<a href="https://colab.research.google.com/github/klausgottlieb/crut-monte-carlo-replication/blob/main/notebook_04_payout_7520.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 04 — Payout Rate and §7520 Rate
## *When Do Charitable Remainder Unitrusts Outperform? A Monte Carlo Analysis*
### Klaus Gottlieb, JD, MS, MBA — Wealth Care Lawyer, Cayucos, CA

---

## Purpose

This notebook examines two parameters that govern the CRUT's internal economics at inception:

### The Payout Rate

The payout rate is the single most consequential *design choice* available to the practitioner drafting the trust instrument. Unlike expected return or longevity — which are uncertain at inception — the payout rate is selected deliberately and fixed irrevocably in the trust document. Getting it right matters.

The payout rate is unusual among CRUT parameters because it has **two simultaneous and opposing effects on the deduction:**

1. **Higher payout → more distributions** — more income to beneficiaries, which is the primary purpose of the trust. This directly increases the economic value of the CRUT to the donor.

2. **Higher payout → smaller charitable deduction** — as the payout rate rises, the IRS remainder factor R falls (because more is expected to go to beneficiaries, less to charity). At a payout rate of approximately 8–9%, the 10% remainder test (IRC §664(d)(2)(D)) becomes binding and caps the payout rate entirely for most age combinations.

The result is a **non-monotonic relationship** between payout rate and win probability. There is an optimal payout rate that maximizes CRUT attractiveness — and it is not necessarily the highest rate the IRS permits.

The IRS imposes a **minimum payout rate of 5%** (IRC §664(d)(2)(A)) and a maximum determined by the 10% remainder test. For Two Life 63/65 at a 5% §7520 rate, the maximum payout rate that passes the 10% test is approximately **9.9%** (the corrected actuarial engine yields a higher remainder factor R at any given payout rate than the prior erroneous engine, which substantially relaxes the 10% constraint). This notebook identifies the win-probability-maximizing rate within that range.

### The §7520 Rate

The §7520 rate ranked **#12 of 12 parameters** in the OAT sensitivity analysis — dead last, with essentially zero impact on win probability across its full historical range (1.2% to 8.2%).

This result is counterintuitive only if one misunderstands the role the §7520 rate plays in CRUT economics. The correct understanding, established in Gottlieb (2025), is as follows.

**The §7520 rate's sole channel of influence is the deduction computation at inception.** It enters through the F-Factor (Equation 1 of Reg. §1.664-4(e)(6)(ii)), which converts the stated payout rate into the adjusted payout rate u. The remainder factor R is then derived from u via the mortality summation. Once R is computed and the deduction is fixed, the §7520 rate plays no further role in the economics of the trust. In particular, it is *not* the discount rate used to present-value the donor's future distributions — that is a separate parameter (`pv_rate`) representing the donor's own time preference, which is independent of the §7520 rate.

The reason the §7520 rate ranks last is therefore a matter of proportions, not cancellation:

1. **The F-Factor is relatively insensitive to §7520 across its realistic range.** As the rate moves from 1.2% to 8.2%, the F-Factor changes modestly (end-of-period quarterly payments), producing a correspondingly modest change in u, R, and the deduction.

2. **The deduction is a small fraction of total CRUT value.** At the baseline, pv_tax ≈ $104K against total mean CRUT wealth of ≈ $567K — roughly 18%. A modest change to a component that represents 18% of total value produces near-zero impact on the win probability comparison.

**Planning implication:** Practitioners do not need to time CRUT formation around the §7520 rate. The common belief that a high §7520 rate is unambiguously better for CRUTs overstates the rate's influence. The deduction does increase with the §7520 rate — but the effect on overall CRUT attractiveness is small. This notebook quantifies exactly how small.

---

## OAT Results from Notebook 01 (Reference)

| Parameter | OAT rank | Win prob range | Range (pp) |
|---|---|---|---|
| Expected return μ | #1 | varies | ~55 |
| Longevity adj | #2 | varies | ~48 |
| Benchmark fee | #3 | varies | ~31 |
| Asset basis | #5 | varies | ~27 |
| **Payout rate** | **#4** | **varies** | **~29** |
| **§7520 rate** | **#12** | **~flat** | **~0** |

The payout rate ranks #4 and the §7520 rate ranks #12. Their contrasting rankings motivate the detailed decomposition in this notebook.

---

## Sections and Figures

1. **Payout rate sweep** — win probability vs. payout rate (5%–8%) across four turnover levels. Highlights the non-monotonic shape and the 10% test constraint.
2. **Payout rate deduction decomposition** — shows deduction, PV tax benefit, and distribution stream PV separately as functions of payout rate.
3. **Payout rate × basis interaction heatmap** — does the optimal payout rate depend on asset basis?
4. **Payout rate × age interaction** — does the optimal payout rate depend on donor age?
5. **§7520 rate sweep and decomposition** — why the rate ranks last despite affecting deduction size.
6. **§7520 rate × payout rate joint analysis** — the combined planning surface.

---

In [ ]:
import subprocess, sys
subprocess.check_call([sys.executable, '-m', 'pip', 'install',
                       'numpy', 'matplotlib', 'scipy', '--quiet'])
print('Dependencies confirmed.')

In [ ]:
from dataclasses import dataclass, replace
from typing import Optional
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

plt.rcParams.update({
    'figure.dpi': 120,
    'font.family': 'sans-serif',
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'figure.facecolor': 'white',
})

TURNOVER_LEVELS = [0.00, 0.20, 0.40, 0.60]
TURNOVER_LABELS = ['0% (buy-and-hold)', '20% (moderate)', '40% (active)', '60% (original baseline)']
TURNOVER_COLORS = ['#08306b', '#2171b5', '#fd8d3c', '#d73027']

print('Imports complete.')

---
## Actuarial Valuation Engine

This notebook uses a self-contained copy of the corrected actuarial engine
first established in Notebook 00 v3. The engine implements the exact
summation method authorized under Treas. Reg. §1.664-4(e)(5)(i), as
documented in:

> Klaus Gottlieb, *From Myth to Math: A Tutorial on the Actuarial Valuation
> of Charitable Remainder Unitrusts under I.R.C. § 7520* (2025) (unpublished
> manuscript), *available at* https://ssrn.com/abstract=5924942.

The F-factor is computed per Reg. §1.664-4(e)(6)(ii) (Equation 1); the
remainder factor via the full probability-weighted summation over Table 2010CM
(Equations 4 and 6). See Notebook 00 for the complete derivation and
validation against CalCRUT.com.

**NB04-specific note:** The `run_simulation()` function in this notebook
preserves the `return_components=True` keyword, which returns mean PV of
distributions, mean CRUT and benchmark wealth, and F-factor values needed
for the §7520 rate decomposition in Section 5.


In [ ]:
# =============================================================================
# ACTUARIAL ENGINE — self-contained copy from Notebook 00 v3
# implements Gottlieb (2025) Equations 1–6 and Table 2010CM
# =============================================================================

# =============================================================================
# ACTUARIAL ENGINE — implements Gottlieb (2025) Equations 1–6 and Table 2010CM
# =============================================================================

from typing import Dict

# Table 2010CM — lx values (number living at age x, radix 100,000).
# Prescribed mortality table under IRC §7520; effective June 1, 2023.
# Source: Treas. Reg. §§20.2031-7, 25.2512-5.
MORTALITY_2010CM: Dict[int, float] = {
    0: 100000.0, 1: 99382.28, 2: 99341.16, 3: 99313.80, 4: 99292.72,
    5: 99276.45, 6: 99261.55, 7: 99248.33, 8: 99236.50, 9: 99226.09,
    10: 99217.03, 11: 99208.80, 12: 99199.98, 13: 99188.21, 14: 99170.64,
    15: 99145.34, 16: 99111.91, 17: 99070.69, 18: 99021.50, 19: 98964.16,
    20: 98898.61, 21: 98824.20, 22: 98741.32, 23: 98652.16, 24: 98559.87,
    25: 98466.80, 26: 98373.71, 27: 98280.09, 28: 98185.51, 29: 98089.05,
    30: 97989.90, 31: 97887.47, 32: 97781.58, 33: 97672.13, 34: 97559.20,
    35: 97442.53, 36: 97321.14, 37: 97193.66, 38: 97058.84, 39: 96915.25,
    40: 96761.20, 41: 96595.51, 42: 96416.30, 43: 96220.61, 44: 96005.41,
    45: 95768.60, 46: 95509.98, 47: 95229.06, 48: 94923.45, 49: 94589.88,
    50: 94225.50, 51: 93828.33, 52: 93398.01, 53: 92934.52, 54: 92438.08,
    55: 91907.95, 56: 91342.02, 57: 90737.24, 58: 90090.97, 59: 89401.06,
    60: 88665.95, 61: 87883.66, 62: 87051.88, 63: 86167.86, 64: 85226.77,
    65: 84221.59, 66: 83142.34, 67: 81978.28, 68: 80728.83, 69: 79387.95,
    70: 77957.53, 71: 76429.84, 72: 74797.63, 73: 73049.33, 74: 71177.55,
    75: 69174.83, 76: 67044.59, 77: 64773.93, 78: 62366.05, 79: 59795.50,
    80: 57080.84, 81: 54213.71, 82: 51205.27, 83: 48059.88, 84: 44808.51,
    85: 41399.79, 86: 37895.25, 87: 34313.98, 88: 30700.82, 89: 27106.68,
    90: 23586.75, 91: 20198.02, 92: 16996.17, 93: 14032.08, 94: 11348.23,
    95: 8975.661, 96: 6931.559, 97: 5218.261, 98: 3823.642, 99: 2722.994,
    100: 1882.108, 101: 1261.083, 102: 818.2641, 103: 513.7236,
    104: 311.8784, 105: 183.0200, 106: 103.8046, 107: 56.91106,
    108: 30.17214, 109: 15.47804, 110: 0.0
}
_MAX_AGE = 110

# ── Mortality helpers ─────────────────────────────────────────────────────────

def _lx(age: int) -> float:
    """lx from Table 2010CM; returns 0 for age >= 110."""
    if age >= _MAX_AGE:
        return 0.0
    return MORTALITY_2010CM.get(int(age), 0.0)

def _tqx(x: int, t: int) -> float:
    """Cumulative probability that a person aged x dies within t years.
    tqx = 1 - l(x+t) / lx.  Returns 1.0 if lx = 0."""
    lx = _lx(x)
    if lx == 0.0:
        return 1.0
    lxt = _lx(x + t) if (x + t) < _MAX_AGE else 0.0
    return 1.0 - lxt / lx

def life_expectancy_single(age: int) -> float:
    """Curtate single-life expectancy from 2010CM: E[K] = sum_{t=1}^{omega} tpx."""
    lx = _lx(age)
    if lx == 0.0:
        return 0.0
    return sum(_lx(age + t) / lx for t in range(1, _MAX_AGE - age + 1))

def life_expectancy_joint_last(age1: int, age2: int) -> float:
    """Curtate last-survivor expectancy from 2010CM.
    E[T_last] = sum_{t=1}^{omega} tpxy, where tpxy = 1 - tqx * tqy
    (Equation 5: law of complementary events)."""
    max_t = _MAX_AGE - min(age1, age2)
    return sum(1.0 - _tqx(age1, t) * _tqx(age2, t) for t in range(1, max_t + 1))

# ── Remainder factor summations ───────────────────────────────────────────────

def _remainder_single(u: float, age: int) -> float:
    """Single-life remainder factor via the Treasury Figure 1 summation
    (Equation 4). Synthetic rate i\'= u/(1-u), decay factor v\'= 1-u.
    R = (1 + i\'/2) * sum_{t=0}^{omega-x} (v\')^{t+1} * (t+1_qx - t_qx)."""
    if u <= 0.0: return 1.0
    if u >= 1.0: return 0.0
    i_syn = u / (1.0 - u)
    v = 1.0 - u
    total, prev_tqx = 0.0, 0.0
    for t in range(_MAX_AGE - age):
        cur_tqx = _tqx(age, t + 1)
        total  += (v ** (t + 1)) * (cur_tqx - prev_tqx)
        prev_tqx = cur_tqx
    return (1.0 + i_syn / 2.0) * total

def _remainder_two_life(u: float, age1: int, age2: int) -> float:
    """Two-life last-survivor remainder factor via the Treasury Figure 1
    summation (Equation 6).
    R = (1 + i\'/2) * sum_{t=0}^{omega} (v\')^{t+1} * (tpxy - t+1_pxy),
    where tpxy = 1 - tqx * tqy  (Equation 5)."""
    if u <= 0.0: return 1.0
    if u >= 1.0: return 0.0
    i_syn = u / (1.0 - u)
    v = 1.0 - u
    max_t = _MAX_AGE - min(age1, age2)
    total, prev_tpxy = 0.0, 1.0
    for t in range(max_t):
        t1pxy  = 1.0 - _tqx(age1, t + 1) * _tqx(age2, t + 1)
        total += (v ** (t + 1)) * (prev_tpxy - t1pxy)
        prev_tpxy = t1pxy
    return (1.0 + i_syn / 2.0) * total

# ── Deduction computation ─────────────────────────────────────────────────────

def compute_deduction(fmv, payout_rate, rate_7520, life_type,
                      age1=65, age2=None, term_years=20,
                      freq=4, lag_months=0, longevity_adj=0):
    """
    Compute CRUT charitable deduction per Treas. Reg. §1.664-4.

    F-factor (Reg §1.664-4(e)(6)(ii), Equation 1):
        F = (1/p) * i * v^(1 + lag_months/12) * (1+i)^(1/p) / ((1+i)^(1/p) - 1)
    where v = 1/(1+i).  For standard end-of-period payments with no
    additional lag (lag_months=0) this reproduces IRS Table F values.

    longevity_adj modifies only the simulation horizon, not the deduction.
    The deduction is fixed at trust inception per the IRS actuarial duration.
    """
    i = rate_7520
    v = 1.0 / (1.0 + i)
    # Equation 1: F-factor
    table_f = (1.0 / freq) * i * (v ** (1.0 + lag_months / 12.0)) *               (1.0 + i) ** (1.0 / freq) / ((1.0 + i) ** (1.0 / freq) - 1.0)
    u = payout_rate * table_f                          # Equation 2

    if life_type == 'Term of Years':
        irs_duration = float(term_years)
        R = max(0.0, min(1.0, (1.0 - u) ** irs_duration))   # Equation 3

    elif life_type == 'Single Life':
        irs_duration = life_expectancy_single(age1)
        R = _remainder_single(u, age1)                        # Equation 4

    else:  # Two Life
        irs_duration = life_expectancy_joint_last(age1, age2)
        R = _remainder_two_life(u, age1, age2)                # Equation 6

    return {
        'deduction':        fmv * R,
        'remainder_factor': R,
        'compliance':       R >= 0.10,
        'irs_duration':     irs_duration,
        'sim_horizon':      irs_duration + longevity_adj,
        'table_f':          table_f,
        'adjusted_payout':  u,
    }

# ── Return path generator (unchanged) ────────────────────────────────────────

def generate_return_paths(mu, sigma, n_years, n_paths, seed=None):
    if seed is not None:
        np.random.seed(seed)
    mu_log    = np.log(1 + mu) - 0.5 * (sigma / (1 + mu)) ** 2
    sigma_log = sigma / (1 + mu)
    return np.exp(np.random.normal(mu_log, sigma_log, size=(n_paths, n_years)))

# ── Scenario parameters ───────────────────────────────────────────────────────

@dataclass
class ScenarioParams:
    fmv:                float         = 1_000_000
    basis_pct:          float         = 0.20
    agi:                float         = 500_000
    payout_rate:        float         = 0.06
    life_type:          str           = 'Two Life'
    age1:               int           = 63
    age2:               Optional[int] = 65
    term_years:         int           = 20
    freq:               int           = 4
    lag_months:         int           = 0
    longevity_adj:      int           = 0
    rate_7520:          float         = 0.05
    pv_rate:            float         = 0.05
    fed_ordinary:       float         = 0.37
    fed_ltcg:           float         = 0.20
    niit:               float         = 0.038
    state_rate:         float         = 0.093
    agi_limit_pct:      float         = 0.30
    carryforward_years: int           = 5
    trust_fee:          float         = 0.01
    bench_fee:          float         = 0.01
    turnover:           float         = 0.20
    mu:                 float         = 0.07
    sigma:              float         = 0.12
    n_paths:            int           = 10_000
    seed:               int           = 42

# ── Simulation engine (unchanged) ────────────────────────────────────────────

def run_simulation(params, return_components=False):
    """
    Run paired-path Monte Carlo CRUT vs. hold-liquidation benchmark.

    If return_components=True, also return the PV of distributions and
    component values separately (used for the §7520 decomposition in
    Section 5 and the payout rate decomposition in Section 2).

    Note on §7520 rate:
    The §7520 rate enters the simulation exclusively through compute_deduction(),
    where it determines the F-factor (Equation 1), the adjusted payout rate u,
    and therefore the remainder factor R and the charitable deduction. After
    compute_deduction() returns, the §7520 rate plays no further role.
    Per Gottlieb (2025) §4.1 and §4.4, the rate's cancellation of growth and
    discount occurs entirely within the IRS regulatory valuation framework —
    it does not reappear as a discount rate for the simulation's cash flows.

    Note on pv_rate:
    pv_rate is the donor's personal discount rate for present-valuing future
    income streams. It is an independent economic parameter with no structural
    relationship to rate_7520. Both default to 5% by convention, but they are
    conceptually distinct and are never co-varied in this analysis.
    """
    p       = params
    tau_ord = p.fed_ordinary + p.state_rate
    tau_cg  = p.fed_ltcg + p.niit + p.state_rate
    # OBBBA: deduction benefit capped at 35% for top-bracket filers
    combined_ord = min(p.fed_ordinary, 0.35) + p.state_rate

    ded_res = compute_deduction(
        fmv=p.fmv, payout_rate=p.payout_rate, rate_7520=p.rate_7520,
        life_type=p.life_type, age1=p.age1, age2=p.age2,
        term_years=p.term_years, freq=p.freq, lag_months=p.lag_months,
        longevity_adj=p.longevity_adj,
    )
    T            = max(1, int(round(ded_res['sim_horizon'])))
    deduction    = ded_res['deduction']
    annual_limit = p.agi * p.agi_limit_pct
    remaining    = deduction
    pv_tax       = 0.0
    for yr in range(p.carryforward_years + 1):
        usable    = min(remaining, annual_limit)
        if usable <= 0:
            break
        pv_tax   += usable * combined_ord / (1 + p.pv_rate) ** yr
        remaining -= usable

    returns = generate_return_paths(p.mu, p.sigma, T, p.n_paths, seed=p.seed)

    # CRUT
    crut_v = np.full(p.n_paths, p.fmv)
    dists  = np.zeros((p.n_paths, T))
    for t in range(T):
        v           = crut_v * (1 - p.trust_fee) * returns[:, t]
        d           = v * p.payout_rate
        dists[:, t] = d * (1 - tau_ord)
        crut_v      = np.maximum(0, v - d)
    disc        = np.array([(1 + p.pv_rate) ** -(t + 1) for t in range(T)])
    pv_dists    = (dists * disc).sum(axis=1)
    crut_wealth = pv_dists + pv_tax

    # Benchmark — hold-liquidation
    bench_v     = np.full(p.n_paths, p.fmv)
    bench_basis = p.fmv * p.basis_pct
    for t in range(T):
        b           = bench_v * (1 - p.bench_fee) * returns[:, t]
        gain        = np.maximum(0, b - bench_basis)
        tax_drag    = p.turnover * gain * tau_cg
        bench_v     = np.maximum(0, b - tax_drag)
        bench_basis = bench_basis + p.turnover * gain * (1 - tau_cg)
    term_gain    = np.maximum(0, bench_v - bench_basis)
    bench_term   = bench_v - term_gain * tau_cg
    bench_wealth = bench_term / (1 + p.pv_rate) ** T

    delta = crut_wealth - bench_wealth
    out = {
        'win_prob':         float(np.mean(delta > 0)),
        'median_delta':     float(np.median(delta)),
        'delta_wealth':     delta,
        'pv_tax':           pv_tax,
        'deduction':        deduction,
        'remainder_factor': ded_res['remainder_factor'],
        'compliance':       ded_res['compliance'],
        'T':                T,
        'params':           p,
    }
    if return_components:
        out['pv_dists']        = float(np.mean(pv_dists))
        out['mean_crut']       = float(np.mean(crut_wealth))
        out['mean_bench']      = float(np.mean(bench_wealth))
        out['table_f']         = ded_res['table_f']
        out['adjusted_payout'] = ded_res['adjusted_payout']
    return out


def bootstrap_ci(data, stat_fn=None, n_boot=1000, ci=0.95, seed=0):
    if stat_fn is None:
        stat_fn = lambda x: np.mean(x > 0)
    rng  = np.random.RandomState(seed)
    n    = len(data)
    boot = [stat_fn(rng.choice(data, size=n, replace=True)) for _ in range(n_boot)]
    alpha = 1 - ci
    return (float(np.percentile(boot, 100 * alpha / 2)),
            float(np.percentile(boot, 100 * (1 - alpha / 2))))


# --- Actuarial verification ---
baseline = ScenarioParams()
_dc = compute_deduction(1_000_000, 0.06, 0.05, 'Two Life', 63, 65, freq=4)
print('Actuarial verification (Two Life 63/65, 6% payout, 5% §7520, quarterly):')
print(f'  Remainder factor : {_dc["remainder_factor"]:.6f}  (CalCRUT: 0.240220)')
print(f'  Deduction on $1M : ${_dc["deduction"]:,.2f}  (CalCRUT: $240,220.39)')
print(f'  IRS duration     : {_dc["irs_duration"]:.2f} yr')
print()
r0 = run_simulation(baseline, return_components=True)
print(f'Baseline (Two Life 63/65, 6% payout, 5% §7520, 20% turnover, n=10,000):')
print(f'  Win probability:   {r0["win_prob"]:.1%}')
print(f'  Median delta:      ${r0["median_delta"]:,.0f}')
print(f'  Deduction:         ${r0["deduction"]:,.0f}')
print(f'  Remainder factor:  {r0["remainder_factor"]:.4f}')
print(f'  10% test:          {r0["compliance"]}')
print(f'  PV tax benefit:    ${r0["pv_tax"]:,.0f}')
print(f'  Mean PV dists:     ${r0["pv_dists"]:,.0f}')
print(f'  Sim horizon:       {r0["T"]} yr')
print()

# 10% test feasibility with corrected engine
print('10% test feasibility across payout rates (Two Life 63/65, 5% §7520):')
for pr in [0.05, 0.06, 0.07, 0.075, 0.079, 0.08, 0.085, 0.090, 0.095]:
    d = compute_deduction(1_000_000, pr, 0.05, 'Two Life', 63, 65)
    status = 'PASS' if d['compliance'] else 'FAIL'
    print(f'  {pr*100:.1f}%  R={d["remainder_factor"]:.4f}  '
          f'deduction=${d["deduction"]:>10,.0f}  10%% test: {status}')
print()
print('Engine ready.')


---
## Section 1 — Payout Rate Sweep

### Figure 1: Win Probability vs. Payout Rate

The payout rate sweep runs from the IRS minimum of **5%** to approximately **8%**, which is near the maximum that passes the 10% remainder test for Two Life 63/65 at a 5% §7520 rate. Rates above the 10% test threshold are shown as a shaded region to indicate they are not permissible at these ages.

**Expected shape of the curves:**

- At low payout rates (near 5%), the deduction is large (high remainder factor) but distributions are modest. The total CRUT value is dominated by deduction benefit.
- As payout rate rises, distributions increase — but so does the adjusted payout factor, which compresses the remainder factor and shrinks the deduction. The two effects are approximately offsetting at intermediate rates.
- Near the 10% test ceiling, the deduction approaches zero. Win probability becomes entirely dependent on the distribution stream, which peaks here — but without the deduction benefit, the CRUT may not outperform.

This produces a curve with an interior maximum. The location of that maximum is the **win-probability-optimizing payout rate** — the answer to the question: "What payout rate should I put in this trust document?"

Note that the win-probability-maximizing rate may differ from the rate that maximizes expected distributions or the rate that maximizes PV of total wealth. Section 2 decomposes these components separately.

In [ ]:
# --- Figure 1: Win probability vs. payout rate ---

payout_rates = np.linspace(0.050, 0.099, 50)

# Find 10% test boundary for this age/rate combination
test_boundary = None
for pr in payout_rates:
    d = compute_deduction(1_000_000, pr, baseline.rate_7520, 'Two Life',
                          baseline.age1, baseline.age2)
    if not d['compliance']:
        test_boundary = pr
        break

print('Running payout rate sweep...')
payout_results = {}  # turnover -> {'wp': array, 'compliance': array}
for tv in TURNOVER_LEVELS:
    wp_arr, comp_arr = [], []
    for pr in payout_rates:
        r = run_simulation(replace(baseline, payout_rate=pr, turnover=tv))
        wp_arr.append(r['win_prob'])
        comp_arr.append(r['compliance'])
    payout_results[tv] = {
        'wp':         np.array(wp_arr),
        'compliance': np.array(comp_arr),
    }
    print(f'  {tv*100:.0f}% turnover: done')

fig, ax = plt.subplots(figsize=(12, 6))

# Shade the region that fails the 10% test
if test_boundary is not None:
    ax.axvspan(test_boundary * 100, payout_rates[-1] * 100,
               alpha=0.10, color='red', label='10% test fails (not permissible)')
    ax.axvline(test_boundary * 100, color='red', lw=1.5, ls='--',
               label=f'10% test boundary ({test_boundary*100:.1f}%)')

for tv, color, tv_label, lw in zip(
        TURNOVER_LEVELS, TURNOVER_COLORS, TURNOVER_LABELS, [2.5,2.5,1.8,1.8]):
    wp = payout_results[tv]['wp']
    ax.plot(payout_rates * 100, wp * 100, color=color, lw=lw,
            label=f'{tv*100:.0f}% turnover')
    # Mark the optimal payout rate
    comp_mask = payout_results[tv]['compliance']
    valid_wp  = np.where(comp_mask, wp, -np.inf)
    opt_idx   = np.argmax(valid_wp)
    ax.scatter(payout_rates[opt_idx]*100, wp[opt_idx]*100,
               color=color, s=80, zorder=5, marker='*')

ax.axvline(6.0, color='orange', lw=1.5, ls=':', label='Baseline (6%)')
ax.axhline(50,  color='black',  lw=1.2, ls=':', label='50% threshold')

ax.set_xlabel('Annual Payout Rate (%)', fontsize=11)
ax.set_ylabel('Win Probability (%)', fontsize=11)
ax.set_xlim(payout_rates[0]*100, payout_rates[-1]*100)
ax.set_ylim(0, 100)
ax.set_title(
    'Figure 1: Win Probability vs. Payout Rate — Four Turnover Levels\n'
    'Two Life 63/65 | 5% §7520 rate | Stars = win-probability-maximizing rate\n'
    'Red shading = rates that fail IRC §664 10%% remainder test (not permissible)',
    fontsize=10
)
ax.legend(fontsize=9)
plt.tight_layout()
plt.savefig('fig1_payout_sweep.png', bbox_inches='tight', dpi=150)
plt.show()

print('Figure 1 saved.')
print()
print('Optimal (win-prob-maximizing) payout rate by turnover:')
for tv, tv_label in zip(TURNOVER_LEVELS, TURNOVER_LABELS):
    wp   = payout_results[tv]['wp']
    comp = payout_results[tv]['compliance']
    valid_wp = np.where(comp, wp, -np.inf)
    opt_idx  = np.argmax(valid_wp)
    print(f'  {tv_label}: {payout_rates[opt_idx]*100:.1f}%  '
          f'(win prob = {wp[opt_idx]:.1%})')

---
## Section 2 — Payout Rate Decomposition

### Figure 2: Component Decomposition — Deduction vs. Distribution Stream

Win probability is the primary output, but understanding *why* it has an interior maximum requires looking at the components separately. This figure shows how the three components of CRUT economic value change as a function of payout rate:

1. **PV of tax benefit from charitable deduction** — decreasing in payout rate (higher payout compresses the deduction).
2. **Mean PV of after-tax distributions** — increasing then flat in payout rate (more distributions, but each dollar of distribution is worth less as the trust corpus depletes faster).
3. **Total CRUT value** — the sum, which has an interior maximum.

The figure also shows the **remainder factor R** on a secondary axis, directly illustrating how the IRS formula links payout rate to the deduction constraint.

This decomposition is important for the manuscript because it explains the non-intuitive finding that increasing the payout rate does not monotonically increase CRUT attractiveness. Practitioners who routinely set payout rates at the IRS maximum (to maximize client income) may be inadvertently reducing the CRUT's competitive advantage.

In [ ]:
# --- Figure 2: Payout rate component decomposition ---
# Run at 20% turnover (primary reference baseline)

pv_tax_arr    = []
pv_dists_arr  = []
total_arr     = []
R_arr         = []
deduction_arr = []
compliance_arr= []

for pr in payout_rates:
    r = run_simulation(replace(baseline, payout_rate=pr, turnover=0.20),
                       return_components=True)
    pv_tax_arr.append(r['pv_tax'])
    pv_dists_arr.append(r['pv_dists'])
    total_arr.append(r['mean_crut'])
    R_arr.append(r['remainder_factor'])
    deduction_arr.append(r['deduction'])
    compliance_arr.append(r['compliance'])

pv_tax_arr    = np.array(pv_tax_arr)    / 1000
pv_dists_arr  = np.array(pv_dists_arr)  / 1000
total_arr     = np.array(total_arr)     / 1000
R_arr         = np.array(R_arr)
compliance_arr= np.array(compliance_arr)
pr_pct        = payout_rates * 100

fig, axes = plt.subplots(2, 1, figsize=(12, 10),
                          gridspec_kw={'height_ratios': [1.6, 1.0]})

# Top panel: value components
ax = axes[0]
ax.stackplot(pr_pct, pv_tax_arr, pv_dists_arr,
             labels=['PV of deduction tax benefit', 'Mean PV of after-tax distributions'],
             colors=['#9ecae1', '#2171b5'], alpha=0.75)
ax.plot(pr_pct, total_arr, color='black', lw=2.5, ls='--',
        label='Total mean CRUT value')

# Mark optimal total
valid_total = np.where(compliance_arr, total_arr, -np.inf)
opt_idx     = np.argmax(valid_total)
ax.axvline(pr_pct[opt_idx], color='green', lw=2, ls=':',
           label=f'Max total value at {pr_pct[opt_idx]:.1f}%')
ax.axvline(6.0, color='orange', lw=1.5, ls=':', label='Baseline (6%)')

if test_boundary is not None:
    ax.axvspan(test_boundary*100, pr_pct[-1], alpha=0.08, color='red')
    ax.axvline(test_boundary*100, color='red', lw=1.5, ls='--',
               label='10% test boundary')

ax.set_ylabel('Mean Present Value ($000)', fontsize=11)
ax.set_title(
    'Figure 2a: CRUT Value Components vs. Payout Rate (20% turnover)\n'
    'Stacked: deduction benefit (light blue) + distribution PV (dark blue)\n'
    'As payout rises, deduction shrinks and distributions grow — the peak total is the optimal rate',
    fontsize=10
)
ax.legend(fontsize=9, loc='upper left')
ax.set_xticklabels([])

# Bottom panel: remainder factor R
ax2 = axes[1]
ax2.plot(pr_pct, R_arr, color='#d73027', lw=2.5,
         label='Remainder factor R')
ax2.axhline(0.10, color='red', lw=1.5, ls='--',
            label='10% minimum (IRC §664)')
ax2.fill_between(pr_pct, R_arr, 0.10,
                 where=(R_arr >= 0.10), alpha=0.15, color='green',
                 label='Permissible zone')
ax2.axvline(6.0, color='orange', lw=1.5, ls=':')
if test_boundary is not None:
    ax2.axvspan(test_boundary*100, pr_pct[-1], alpha=0.08, color='red')
    ax2.axvline(test_boundary*100, color='red', lw=1.5, ls='--')

ax2.set_xlabel('Annual Payout Rate (%)', fontsize=11)
ax2.set_ylabel('Remainder Factor R', fontsize=11)
ax2.set_title(
    'Figure 2b: Remainder Factor R vs. Payout Rate\n'
    'R must be ≥ 10% for the trust to qualify as a charitable remainder trust (IRC §664(d)(2)(D))',
    fontsize=10
)
ax2.legend(fontsize=9)
ax2.set_xlim(pr_pct[0], pr_pct[-1])

plt.tight_layout()
plt.savefig('fig2_payout_decomposition.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 2 saved.')
print()
print(f'Peak total CRUT value at payout rate: {pr_pct[opt_idx]:.1f}%')
print(f'  PV tax benefit at {pr_pct[opt_idx]:.1f}%: ${pv_tax_arr[opt_idx]:,.0f}K')
print(f'  PV distributions at {pr_pct[opt_idx]:.1f}%: ${pv_dists_arr[opt_idx]:,.0f}K')
print(f'  Total at {pr_pct[opt_idx]:.1f}%: ${total_arr[opt_idx]:,.0f}K')
print(f'  Remainder factor R: {R_arr[opt_idx]:.4f}')

---
## Section 3 — Payout Rate × Basis Interaction

### Figure 3: Heatmap — Win Probability vs. Payout Rate and Basis

The asset basis fraction and the payout rate both affect CRUT win probability, but potentially in different directions depending on their combination. This heatmap answers the question: **does the optimal payout rate depend on how appreciated the contributed asset is?**

**Economic reasoning for a potential interaction:**
- At low basis (highly appreciated asset), the CRUT offers a large immediate benefit — avoiding capital gains tax on the embedded gain. The deduction compounds this benefit. In this regime, any payout rate that passes the 10% test may produce a favorable CRUT.
- At high basis (minimally appreciated asset), the benchmark avoids very little capital gains tax on liquidation. The CRUT must rely almost entirely on the deduction and distribution advantages. In this regime, the payout rate choice matters more.

If the heatmap's 50% contour shifts horizontally as payout rate changes, it confirms that the optimal payout rate and the break-even basis level are interrelated planning variables.

In [ ]:
# --- Figure 3: Payout rate x basis heatmap ---

basis_grid  = np.linspace(0.02, 0.80, 20)
payout_grid = np.linspace(0.050, 0.079, 15)  # stay within 10% test

hm_20 = np.zeros((len(payout_grid), len(basis_grid)))  # 20% turnover
hm_60 = np.zeros((len(payout_grid), len(basis_grid)))  # 60% turnover
hm_comp = np.ones((len(payout_grid), len(basis_grid)), dtype=bool)  # 10% test

print('Running payout x basis heatmap (20 x 15 x 2 = 600 simulations)...')
for i, pr in enumerate(payout_grid):
    for j, bv in enumerate(basis_grid):
        p20 = replace(baseline, payout_rate=pr, basis_pct=bv, turnover=0.20)
        p60 = replace(baseline, payout_rate=pr, basis_pct=bv, turnover=0.60)
        r20 = run_simulation(p20)
        r60 = run_simulation(p60)
        hm_20[i, j]   = r20['win_prob']
        hm_60[i, j]   = r60['win_prob']
        hm_comp[i, j] = r20['compliance']  # same compliance for both

fig, axes = plt.subplots(1, 2, figsize=(16, 7))

for ax, hm, tv_label in [
    (axes[0], hm_20, '20% Turnover'),
    (axes[1], hm_60, '60% Turnover'),
]:
    # Mask non-compliant cells
    hm_masked = np.ma.array(hm * 100, mask=~hm_comp)
    im = ax.imshow(
        hm_masked, origin='lower', aspect='auto',
        extent=[basis_grid[0]*100, basis_grid[-1]*100,
                payout_grid[0]*100, payout_grid[-1]*100],
        cmap='RdYlBu', vmin=5, vmax=95
    )
    plt.colorbar(im, ax=ax, label='Win Probability (%)')

    b_g, p_g = np.meshgrid(basis_grid*100, payout_grid*100)
    cs = ax.contour(b_g, p_g, hm_masked, levels=[50],
                    colors='black', linewidths=2)
    ax.clabel(cs, fmt='50%%', fontsize=10)

    ax.scatter([20], [6.0], color='orange', s=200, marker='*',
               zorder=5, edgecolors='black',
               label='Baseline (20% basis, 6% payout)')
    ax.set_xlabel('Asset Basis Fraction (% of FMV)', fontsize=11)
    ax.set_ylabel('Annual Payout Rate (%)', fontsize=11)
    ax.set_title(f'Figure 3: Payout Rate × Basis Heatmap\n{tv_label}', fontsize=10)
    ax.legend(fontsize=9)

fig.suptitle(
    'Figure 3: Win Probability — Payout Rate × Asset Basis\n'
    'Black contour = 50% decision boundary | White/grey = rates failing 10%% test\n'
    'Compare contour slope: if vertical, payout and basis are independent; if diagonal, they interact',
    fontsize=10
)
plt.tight_layout()
plt.savefig('fig3_payout_basis_heatmap.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 3 saved.')

---
## Section 4 — Payout Rate × Starting Age

### Figure 4: Optimal Payout Rate as a Function of Age

The 10% test constraint tightens as donor age decreases — younger donors have longer expected trust durations, which means larger total payouts expected, which compresses the remainder factor more quickly. This implies that the **feasible payout range narrows at younger ages** and widens at older ages.

This figure shows, for each starting age combination, the win-probability-maximizing payout rate within the 10% test constraint. It answers a direct practitioner question: **given my client's age, what payout rate should I recommend?**

Age gap is held constant at 2 years throughout.

In [ ]:
# --- Figure 4: Optimal payout rate vs. starting age ---

younger_ages = np.arange(50, 76, 1)
payout_scan  = np.linspace(0.050, 0.095, 45)  # wide enough to find boundary

print('Running payout x age sweep...')

opt_payout_20 = []  # optimal payout rate at 20% turnover
opt_payout_60 = []  # optimal payout rate at 60% turnover
opt_wp_20     = []
opt_wp_60     = []
max_compliant = []  # maximum compliant payout rate
min_R_at_5pct = []  # remainder factor at 5% payout (minimum payout)

for age in younger_ages:
    wp_20_arr, wp_60_arr, comp_arr = [], [], []
    for pr in payout_scan:
        d  = compute_deduction(1_000_000, pr, 0.05, 'Two Life', int(age), int(age+2))
        comp_arr.append(d['compliance'])
        if d['compliance']:
            r20 = run_simulation(replace(baseline, age1=int(age), age2=int(age+2),
                                         payout_rate=pr, turnover=0.20))['win_prob']
            r60 = run_simulation(replace(baseline, age1=int(age), age2=int(age+2),
                                         payout_rate=pr, turnover=0.60))['win_prob']
        else:
            r20, r60 = -1.0, -1.0
        wp_20_arr.append(r20)
        wp_60_arr.append(r60)

    comp_arr  = np.array(comp_arr)
    wp_20_arr = np.array(wp_20_arr)
    wp_60_arr = np.array(wp_60_arr)

    opt_idx_20 = np.argmax(wp_20_arr)
    opt_idx_60 = np.argmax(wp_60_arr)
    opt_payout_20.append(payout_scan[opt_idx_20] if comp_arr[opt_idx_20] else np.nan)
    opt_payout_60.append(payout_scan[opt_idx_60] if comp_arr[opt_idx_60] else np.nan)
    opt_wp_20.append(wp_20_arr[opt_idx_20] if comp_arr[opt_idx_20] else np.nan)
    opt_wp_60.append(wp_60_arr[opt_idx_60] if comp_arr[opt_idx_60] else np.nan)

    compliant_rates = payout_scan[comp_arr]
    max_compliant.append(compliant_rates.max() if len(compliant_rates) > 0 else np.nan)

    d5 = compute_deduction(1_000_000, 0.05, 0.05, 'Two Life', int(age), int(age+2))
    min_R_at_5pct.append(d5['remainder_factor'])

opt_payout_20 = np.array(opt_payout_20) * 100
opt_payout_60 = np.array(opt_payout_60) * 100
max_compliant  = np.array(max_compliant)  * 100
min_R_at_5pct  = np.array(min_R_at_5pct)
print('Done.')

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

ax = axes[0]
ax.fill_between(younger_ages, 5.0, max_compliant,
                alpha=0.12, color='green', label='Permissible payout range')
ax.plot(younger_ages, opt_payout_20, color='#2171b5', lw=2.5,
        label='Optimal rate (20% turnover)')
ax.plot(younger_ages, opt_payout_60, color='#d73027', lw=2.5, ls='--',
        label='Optimal rate (60% turnover)')
ax.plot(younger_ages, max_compliant, color='gray', lw=1.5, ls=':',
        label='Max compliant rate (10% test ceiling)')
ax.axvline(63, color='orange', lw=1.5, ls=':', label='Baseline age (63)')
ax.axhline(6.0, color='orange', lw=0.8, ls=':')
ax.set_xlabel('Younger Beneficiary Age (older is age+2)', fontsize=11)
ax.set_ylabel('Annual Payout Rate (%)', fontsize=11)
ax.set_title('Figure 4a: Optimal and Maximum Payout Rate vs. Age\n'
             'Green band = permissible range | Stars = win-prob-maximizing rate', fontsize=10)
ax.legend(fontsize=8)
ax.set_xlim(50, 75)

ax2 = axes[1]
ax2.plot(younger_ages, np.array(opt_wp_20)*100, color='#2171b5', lw=2.5,
         label='Win prob at optimal rate (20% turnover)')
ax2.plot(younger_ages, np.array(opt_wp_60)*100, color='#d73027', lw=2.5, ls='--',
         label='Win prob at optimal rate (60% turnover)')
ax2.axhline(50, color='black', lw=1.2, ls=':')
ax2.axvline(63, color='orange', lw=1.5, ls=':')
ax2.set_xlabel('Younger Beneficiary Age', fontsize=11)
ax2.set_ylabel('Win Probability at Optimal Payout Rate (%)', fontsize=11)
ax2.set_title('Figure 4b: Maximum Achievable Win Probability vs. Age\n'
              '(using the win-probability-maximizing payout rate)', fontsize=10)
ax2.legend(fontsize=9)
ax2.set_xlim(50, 75)
ax2.set_ylim(0, 100)

fig.suptitle(
    'Figure 4: Optimal Payout Rate and Maximum Win Probability vs. Donor Age\n'
    '2-year age gap held constant | 5% §7520 rate | Baseline: 20% basis',
    fontsize=10
)
plt.tight_layout()
plt.savefig('fig4_optimal_payout_age.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 4 saved.')

# Print summary table
print()
print('Optimal payout rate by age (20% turnover):')
print(f'{"Age pair":<12} {"Opt rate":>9} {"Max rate":>9} {"Win prob":>9}')
print('-' * 44)
for i, age in enumerate(younger_ages):
    if age in [50, 55, 60, 63, 65, 70, 75]:
        print(f'{age}/{age+2:<10} {opt_payout_20[i]:>8.1f}%  '
              f'{max_compliant[i]:>8.1f}%  '
              f'{opt_wp_20[i]*100 if not np.isnan(opt_wp_20[i]) else 0:>8.1f}%')

---
## Section 5 — §7520 Rate: Why It Ranks Last

### Figures 5–6: §7520 Rate Sweep and Single-Channel Decomposition

The §7520 rate's rank of **#12 of 12** with essentially zero win-probability
impact across a 7-percentage-point range (1.2%–8.2%) is the most
counterintuitive finding in the sensitivity analysis. Practitioners widely
believe that higher §7520 rates are unambiguously better for CRUTs.

**The correct mechanism — a single channel, not a cancellation:**

Per Gottlieb (2025) §4.1 and §4.4, the §7520 rate operates through exactly
one channel: the F-Factor computation in `compute_deduction()`. It determines
the adjusted payout rate u, which determines the remainder factor R, which
determines the deduction. Once the deduction is fixed at inception, the §7520
rate is gone from the economics. It is not a discount rate for the simulation's
cash flows — that role belongs to `pv_rate`, an independent donor preference
parameter.

The near-zero OAT sensitivity therefore has a straightforward proportional
explanation with two components:

**Component 1 — F-Factor insensitivity.** As the §7520 rate moves across
its historical range, the F-Factor (end-of-period, quarterly) changes modestly.
This produces a modest change in u, a modest change in R, and a modest change
in the deduction. Figure 6 (left panel) shows the deduction and PV tax benefit
as functions of §7520 rate — the slope is real but shallow.

**Component 2 — Deduction is a small fraction of total CRUT value.**
At the baseline, pv_tax ≈ $104K against mean total CRUT wealth ≈ $567K —
approximately 18%. A modest change to an 18% component produces near-zero
impact on the win probability comparison against a benchmark that has no
corresponding deduction component at all.

**Figure 5** shows the win probability sweep — essentially flat across the
full historical range at all four turnover levels.

**Figure 6** shows the decomposition: left panel traces the deduction and PV
tax benefit vs. §7520 rate (the one real channel); right panel shows deduction
as a share of total CRUT value, illustrating why the channel's modest
sensitivity does not translate into meaningful win probability movement.


In [ ]:
# --- Figures 5-6: §7520 rate sweep and single-channel decomposition ---
#
# Per Gottlieb (2025) §4.1 and §4.4, the §7520 rate operates through exactly
# one channel: the F-Factor in compute_deduction(). It is not a discount rate
# for simulation cash flows. The analysis below reflects this correctly:
#
#   Condition A: rate_7520 varies; pv_rate fixed at 5% (the correct model)
#   This is the only condition. We do NOT vary pv_rate alongside rate_7520,
#   as that would misrepresent the §7520 rate as having a second channel
#   it does not possess.
#
#   The decomposition shows:
#     (1) How the F-Factor and deduction respond to rate_7520
#     (2) The deduction's share of total CRUT value — why a real but modest
#         channel produces near-zero win probability sensitivity

# Historical range of §7520 rates: ~1.2% (2021 low) to ~8.2% (early 2000s)
rate_7520_vals = np.linspace(0.012, 0.082, 25)

print('Running §7520 rate sweep (single-channel: deduction only)...')

wp_20, wp_60   = [], []   # win probability at 20% and 60% turnover
ded_arr        = []       # charitable deduction
pvtax_arr      = []       # PV tax benefit
pvdist_arr     = []       # mean PV of distributions
total_crut_arr = []       # mean total CRUT wealth
table_f_arr    = []       # F-Factor at each rate
u_arr          = []       # adjusted payout rate at each rate
share_arr      = []       # deduction as share of total CRUT value

for r7 in rate_7520_vals:
    # pv_rate held fixed at 5% in all cases — §7520 rate only affects deduction
    p20 = replace(baseline, rate_7520=r7, pv_rate=0.05, turnover=0.20)
    p60 = replace(baseline, rate_7520=r7, pv_rate=0.05, turnover=0.60)
    r20 = run_simulation(p20, return_components=True)
    r60 = run_simulation(p60, return_components=True)
    wp_20.append(r20['win_prob'])
    wp_60.append(r60['win_prob'])
    ded_arr.append(r20['deduction'])
    pvtax_arr.append(r20['pv_tax'])
    pvdist_arr.append(r20['pv_dists'])
    total_crut_arr.append(r20['mean_crut'])
    table_f_arr.append(r20['table_f'])
    u_arr.append(r20['adjusted_payout'])
    share_arr.append(r20['pv_tax'] / r20['mean_crut'] * 100)

print('Done.')

wp_20 = np.array(wp_20) * 100
wp_60 = np.array(wp_60) * 100
r7_pct = rate_7520_vals * 100

# Report key values at low, baseline, and high §7520 rates
idx_lo   = 0
idx_base = np.argmin(np.abs(rate_7520_vals - 0.05))
idx_hi   = len(rate_7520_vals) - 1

print()
print(f"{'§7520 rate':<12} {'F-Factor':>10} {'u':>8} {'Deduction':>12} "
      f"{'pv_tax':>10} {'pv_tax/CRUT':>12} {'Win%(20T)':>10}")
print('-' * 78)
for idx in [idx_lo, idx_base, idx_hi]:
    r7 = rate_7520_vals[idx]
    print(f"  {r7*100:.1f}%      {table_f_arr[idx]:>9.6f}  "
          f"{u_arr[idx]*100:>7.4f}%  "
          f"${ded_arr[idx]:>10,.0f}  "
          f"${pvtax_arr[idx]:>8,.0f}  "
          f"{share_arr[idx]:>10.1f}%  "
          f"{wp_20[idx]:>9.1f}%")

print()
print(f"Win probability range across full §7520 sweep (20% turnover): "
      f"{wp_20.max()-wp_20.min():.1f} pp")
print(f"Win probability range across full §7520 sweep (60% turnover): "
      f"{wp_60.max()-wp_60.min():.1f} pp")
print()
print(f"Deduction range: ${min(ded_arr):,.0f} – ${max(ded_arr):,.0f}  "
      f"(Δ = ${max(ded_arr)-min(ded_arr):,.0f})")
print(f"pv_tax/CRUT share range: {min(share_arr):.1f}% – {max(share_arr):.1f}%")

# ── Figure 5: Win probability sweep ──────────────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(r7_pct, wp_20, color='#2171b5', lw=2.5, label='20% turnover')
ax.plot(r7_pct, wp_60, color='#d73027', lw=2.5, ls='--', label='60% turnover')
ax.axvline(5.0, color='orange', lw=1.5, ls=':', label='Baseline §7520 rate (5%)')
ax.axhline(50,  color='black',  lw=1.2, ls=':')
ax.set_xlabel('§7520 Rate (%)', fontsize=11)
ax.set_ylabel('Win Probability (%)', fontsize=11)
ax.set_title(
    'Figure 5: Win Probability vs. §7520 Rate (pv_rate fixed at 5%)\n'
    '§7520 rate affects only the charitable deduction via the F-Factor — '
    'no second channel exists\n'
    'Near-flat curve reflects modest F-Factor sensitivity × small deduction share of CRUT value',
    fontsize=10
)
ax.legend(fontsize=9)
ax.set_ylim(0, 100)
plt.tight_layout()
plt.savefig('fig5_7520_sweep.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 5 saved.')

# ── Figure 6: Single-channel decomposition ────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Deduction and PV tax benefit vs. §7520 rate
ax = axes[0]
ax.plot(r7_pct, np.array(ded_arr)/1000,   color='#d73027', lw=2.5,
        label='Charitable deduction ($000)')
ax.plot(r7_pct, np.array(pvtax_arr)/1000, color='#2171b5', lw=2.5,
        label='PV tax benefit ($000)')
ax.axvline(5.0, color='orange', lw=1.5, ls=':', label='Baseline (5%)')
ax.set_xlabel('§7520 Rate (%)', fontsize=11)
ax.set_ylabel('Amount ($000)', fontsize=11)
ax.set_title('Deduction and PV Tax Benefit vs. §7520 Rate\n'
             'Real but shallow slope — F-Factor insensitivity across realistic range',
             fontsize=10)
ax.legend(fontsize=9)

# Right: pv_tax as share of total CRUT value — the proportional argument
ax2 = axes[1]
ax2.plot(r7_pct, share_arr, color='#1a9641', lw=2.5)
ax2.axvline(5.0, color='orange', lw=1.5, ls=':', label='Baseline (5%)')
ax2.axhline(share_arr[idx_base], color='gray', lw=1, ls='--',
            label=f'Baseline share: {share_arr[idx_base]:.1f}%')
ax2.set_xlabel('§7520 Rate (%)', fontsize=11)
ax2.set_ylabel('pv_tax as % of Mean CRUT Wealth', fontsize=11)
ax2.set_title('Deduction Share of Total CRUT Value\n'
              '~15–22%: a modest change to a modest component\n'
              '→ near-zero impact on win probability', fontsize=10)
ax2.legend(fontsize=9)
ax2.set_ylim(0, 40)

fig.suptitle(
    'Figure 6: §7520 Rate — Single-Channel Decomposition\n'
    'Left: deduction rises with §7520 rate (real but shallow) | '
    'Right: deduction represents ~15–22% of CRUT value\n'
    'Near-zero win probability sensitivity follows from both factors combined',
    fontsize=10
)
plt.tight_layout()
plt.savefig('fig6_7520_decomposition.png', bbox_inches='tight', dpi=150)
plt.show()
print('Figure 6 saved.')


In [ ]:
# =============================================================================
# FINAL SUMMARY
# =============================================================================

print('=' * 65)
print('NOTEBOOK 04 — KEY FINDINGS')
print('=' * 65)
print()
print('1. Payout rate (Figure 1):')
print('   Win probability is non-monotonic in payout rate.')
print('   Interior maximum exists within the 10%% test constraint.')
for tv, tv_label in zip([0.20, 0.60], ['20% turnover', '60% turnover']):
    wp   = payout_results[tv]['wp']
    comp = payout_results[tv]['compliance']
    valid_wp = np.where(comp, wp, -np.inf)
    opt_idx  = np.argmax(valid_wp)
    print(f'   {tv_label}: optimal payout = {payout_rates[opt_idx]*100:.1f}%  '
          f'(win prob = {wp[opt_idx]:.1%})')
print()
print('2. Payout rate decomposition (Figure 2):')
print(f'   At {pr_pct[opt_idx]:.1f}% payout (20% turnover):')
print(f'     PV deduction benefit: ${pv_tax_arr[opt_idx]:,.0f}K')
print(f'     PV distributions:     ${pv_dists_arr[opt_idx]:,.0f}K')
print(f'     Total CRUT value:     ${total_arr[opt_idx]:,.0f}K')
print()
print('3. §7520 rate (Figures 5-6):')
wp_range_20 = wp_20.max() - wp_20.min()
wp_range_60 = wp_60.max() - wp_60.min()
print(f'   Win prob range (20% turnover): {wp_range_20:.1f} pp  — confirms #12 rank')
print(f'   Win prob range (60% turnover): {wp_range_60:.1f} pp')
print(f'   pv_tax share of CRUT value at baseline: {share_arr[idx_base]:.1f}%')
print('   Near-zero sensitivity = modest F-Factor response × small deduction share.')
print()
print('   PLANNING IMPLICATION: Practitioners need not time CRUT formation')
print('   around the §7520 rate. The common belief that a high §7520 rate')
print('   is unambiguously better for CRUTs is not supported by this analysis.')
print()

import os
figures = ['fig1_payout_sweep.png', 'fig2_payout_decomposition.png',
           'fig3_payout_basis_heatmap.png', 'fig4_optimal_payout_age.png',
           'fig5_7520_sweep.png', 'fig6_7520_decomposition.png']
print('Figure completion check:')
for f in figures:
    print(f"  {'OK' if os.path.exists(f) else 'MISSING'} {f}")
print()
print('Notebook 04 complete. Proceed to Notebook 05 — State Tax Sweep.')